Node 정의

In [1]:
class Node:
    def __init__(self, key, index=0):
        self.parent = None
        self.key = key
        self.index = index
        self.left = None    #None or empty leaf node for red black tree
        self.right = None   
        self.color = 0      #black(1), red(0)
    
    def addChild(self, lr, childNode):
        if lr == 0: #left child
            self.left = childNode 
            childNode.index = self.index * 2 
            childNode.parent = self
            
        elif lr == 1: #right child            
            self.right = childNode 
            childNode.index = self.index * 2 + 1
            childNode.parent = self

        else :
            return 1
        return 0
    
    def readKey(self):
        return self.key
    
    def writeKey(self, key):
        self.key = key


In [2]:
def binarySearchTreeFindNode(node: Node, key, index) :
    
    
    if key == None and index == None:
        return None
    
    if node == None or node.key == key or node.index ==index:
        return node
    
    if key != None:       
        if node.key < key :
            node = binarySearchTreeFindNode(node.right, key, index)
        else :
            node = binarySearchTreeFindNode(node.left, key, index)
            
    elif index != None:
        mask = 1
        numOfRound = 2
        while numOfRound < index :
            if (index & mask) != 0:
                node=node.right
            else:
                node=node.left
            mask = mask << 1
            numOfRound *= 2
    
    return node

 BST에서 노드 추가
 * _ binarySearchTreeInsertNode (parent, key)
   [1] parent: 추가될 노드의 상위 노드. 탐색을 통하여 parent가 NULL인 위치를 찾아, 해당 위치에 노드를 삽입한다.
   [2] key: 삽입 하려는 노드의 키 값
   단,   root node 는 별도로 처리한다.
   
   * binarySearchTreeInsertNode (root, key)
   [1] root: tree의 루트노드
   [2] key: 삽입하려는 노드의 키 값
   

In [3]:
def _binarySearchTreeInsertNode(parent: Node, key) -> (int, Node):    
     
    if parent == None or parent.index == 0:  # if index=0, it is a empty leaf node for red black tree
        return 1, None
    
    #to check whether input type of 'parent' is Node
    #if isinstance(parent,Node) == False:
    #    return 2, None    
    
    node=None
    
    if key < parent.key:
        result, node =  _binarySearchTreeInsertNode(parent.left, key)
        if result == 1:
            node = Node(key)
            parent.addChild(0, node) #add a left child to a parent node
            
    else:
        result, node =  _binarySearchTreeInsertNode(parent.right, key)
        if result == 1:
            node = Node(key)
            parent.addChild(1, node) #add a right child to a parent node
            
    return 0, node


In [4]:

def binarySearchTreeInsertNode(root: Node, key) -> (int, Node):  
    
    if root == None: #generate a root
        node = Node(key, index=1)  #a root is a black node
        return 0, node
    else:
        result, node = _binarySearchTreeInsertNode(root, key)
        return result, node 


BST에서 노드 삭제
: 주어진 키 값을 이용하여 노드를 탐색한 후, 해당 노드를 삭제한다.

In [5]:
def _deleteNode(node):
    if node.left == None and node.right == None:
        return None
    
    elif node.left != None and node.right == None:
        return node.left
    
    elif node.left == None and node.right != None:
        return node.right
    
    else:
        target = node.right 
        while target.left != None:
            parent = target
            target = target.left
        
        #delete target
        node.key = target.key
        
        if target == node.right:
            node.right = target.right
        else:
            parent.right = target.right
        return node 

# it finds a node with removeNode.key and then remove the node from tree with the root    
def binarySearchTreeDeleteNode(root:Node, key, index):
    
    removeNode = binarySearchTreeFindNode(root, key, index)
    
    if removeNode.key == root.key:
        root = _deleteNode(removeNode)
        
    elif removeNode.parent.left != None and removeNode.parent.left.key == removeNode.key :
        removeNode.parent.left = _deleteNode(removeNode)
    
    else:
        removeNode.parent.right = _deleteNode(removeNode)


Test

In [11]:
root=Node(2,1) #root node    
result, node=binarySearchTreeInsertNode(root,3)
result, node=binarySearchTreeInsertNode(root,5)
result, node=binarySearchTreeInsertNode(root,1)
print(root)
node=binarySearchTreeFindNode(root, None, 7)
print(node)
node=binarySearchTreeDeleteNode(root, 1, None)
print(root)